In [1]:
import os
import csv
import json
import time
import datetime
import requests
import datetime
import unicodedata
import dateutil.parser
import pandas as pd

In [2]:
DIR = os.getcwd()

In [3]:
def auth():
    return 'AAAAAAAAAAAAAAAAAAAAADUnUQEAAAAAoGjrMfntgwKfsX2eY8fs54dwl%2Fk%3D5ZlgNgFJGSqg85K3MPwAoeFqjGnXihYOR72KKyEriAGm5D7dTu'
    #return os.getenv('TOKEN')

In [4]:
def create_headers(bearer_token):
    headers = {"Authorization": f"Bearer {bearer_token}"}
    return headers

In [5]:
def create_url(keyword, start_date, end_date, max_results = 100):
    
    search_url = "https://api.twitter.com/2/tweets/search/recent" 
    query_params = {'query': keyword,
                    'start_time': start_date,
                    'end_time': end_date,
                    'max_results': max_results,
                    'tweet.fields': 'id,text',
                    'user.fields': 'verified',
                    'next_token': {}}
    return (search_url, query_params)

In [6]:
def getTime():
    startTime = datetime.date.today().isoformat() + 'T00:00:00Z'
    endTime = datetime.datetime.now().isoformat(timespec='seconds') + 'Z'
    return startTime, endTime

In [7]:
def connect_to_endpoint(url, headers, params, next_token = None):
    params['next_token'] = next_token   
    response = requests.request("GET", url, headers = headers, params = params)
    print("Endpoint Response Code: " + str(response.status_code))
    if response.status_code != 200:
        raise Exception(response.status_code, response.text)
    return response.json()

In [8]:
def makeRequest(keyword):
    bearer_token = auth()
    headers = create_headers(bearer_token)
    startTime, endTime = getTime()
    url = create_url(keyword, startTime,endTime, 100)
    json_response = connect_to_endpoint(url[0], headers, url[1])
    return json_response

In [9]:
def runDaily():
    targets = pd.read_csv(DIR+'/targetSP.csv')
    json_responses = targets.Symbol.apply(lambda x: [x, makeRequest(x + ' lang:en')])
    responses = pd.DataFrame(columns=['target', 'id', 'text'])
    for response in json_responses:  
        target = response[0]
        tweetData = response[1]['data']
        for tweet in tweetData:
            responses = responses.append({'target': target, 'id': tweet['id'], 'text': tweet['text'].replace('\n',' ').replace('\r', ' ').replace('\t', ' ').replace('\f', ' ')}, ignore_index=True)
    if not os.path.exists('data/' +(datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')):
        os.mkdir('data/' +(datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d'))
    responses.to_csv('data/' + (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d') + '/tweets.csv')
    return responses

In [10]:
data = runDaily()

Endpoint Response Code: 200
Endpoint Response Code: 200
Endpoint Response Code: 200
Endpoint Response Code: 200
Endpoint Response Code: 200
Endpoint Response Code: 200
Endpoint Response Code: 200
Endpoint Response Code: 200
Endpoint Response Code: 200
Endpoint Response Code: 200


FileNotFoundError: [Errno 2] No such file or directory: 'data/2021-12-23'